# Description

For this exercise, you will need to create a particular API for working with track athletes at a competition consisting of several different events.  Of course, you should utilize the data structures discussed in this lesson to achieve this goal.  The API you need to follow has several constraints.

* Various medals won correspond to different prize amounts (whatever currency): Gold is 1000, Silver is 500, and Bronze is 250.
* A `Medal` object will allow reference to these different types of medals, but will raise an exception if other attributes are accessed.
* A runner (an instance of `Competition`) will have attributes corresponding to each type of medal, or None if no placement in an event.
* The `Competition` class provides a `.winnings()` method to tally the winnings across events.
* The `Competition` class provides a `.medals()` method to report on medals won, sorted from best to worst.

These are examples of the API you will need to conform with:

```python
runner = Competition()
runner.foot_500m = Medal.Gold
runner.hurdle_100m = Medal.Silver
runner.hurdle_500m = Medal.Bronze
runner.foot_50m = Medal.Silver
```

With that `runner` instance we can call:

```python
>>> runner.winnings()
2250
>>> runner.medals()
[('foot_500m', 'Gold'),
 ('foot_50m', 'Silver'),
 ('hurdle_100m', 'Silver'),
 ('hurdle_400m', 'Bronze'),
 ('foot_100m', 'No Placement')]
```

With a `Medal` object:

```python
>>> try:
...     Medal.Copper
... except AttributeError as err:
...     print(repr(err))
AttributeError('Copper')
```

# Setup

In [1]:
# These classes provides (almost) correct interfaces, not correct behaviors
# Of note: these versions do not depend on config of instances whatsoever

class Medal:
    Gold = 1
    Silver = 2
    Bronze = 3
    
class Competition:
    def winnings(self):
        return 2250
    def medals(self):
        return [
            ('foot_50m', 'Gold'),
            ('foot_500m', 'Gold'),
            ('hurdle_400m', 'Bronze'),
            ('foot_100m', 'No Placement'),
            ('hurdle_100m', 'No Placement')
        ]

# Solution

In [2]:
from dataclasses import dataclass, asdict
from enum import Enum

class Medal(Enum):
    Gold = 1000
    Silver = 500
    Bronze = 250
    
@dataclass
class Competition:
    foot_500m: Medal = None
    foot_100m: Medal = None
    foot_50m: Medal = None
    hurdle_100m: Medal = None
    hurdle_400m: Medal = None
        
    def winnings(self):
        return sum(v.value for v in asdict(self).values() if v)
    
    def medals(self):
        results = []
        for race, place in asdict(self).items():
            results.append((race, place.name if place else "No Placement"))
        order = {'Gold': 1, 'Silver': 2, 'Bronze': 4, 'No Placement': 5}
        results.sort(key=lambda item: order[item[1]])
        return results

# Test Cases

In [3]:
def test_medal1():
    assert Medal.Gold == Medal.Gold
    assert Medal.Silver == Medal.Silver
    assert Medal.Bronze == Medal.Bronze

    try:
        Medal.Copper
        assert False, "There should be no Copper medal"
    except AttributeError:
        pass # All good, raise exception
        
test_medal1()

In [4]:
def test_medal2():
    assert Medal.Gold != Medal.Silver
    assert Medal.Silver != Medal.Bronze
    assert Medal.Bronze != Medal.Gold
    
test_medal2()

In [5]:
def test_api1():
    runner = Competition()
    runner.hurdle_100m = Medal.Bronze
    runner.hurdle_400m = Medal.Silver
    runner.foot_100m = Medal.Gold
    assert runner.winnings() == 1750, "Winnings should be 1750"

test_api1()

In [6]:
def test_api2():
    correct = [('hurdle_100m', 'Gold'), 
               ('hurdle_400m', 'Silver'), 
               ('foot_100m', 'Bronze'), 
               ('foot_50m', 'Bronze'), 
               ('foot_500m', 'No Placement')]
    runner = Competition()
    runner.hurdle_100m = Medal.Gold
    runner.hurdle_400m = Medal.Silver
    runner.foot_50m = Medal.Bronze
    runner.foot_100m = Medal.Bronze
    assert runner.medals() == correct, f"Should be: {correct}"

test_api2()

In [7]:
def test_api3():
    correct = [('foot_100m', 'Gold'),
               ('hurdle_100m', 'Gold'),
               ('hurdle_400m', 'Silver'),
               ('foot_500m', 'No Placement'),
               ('foot_50m', 'No Placement')]
    runner = Competition()
    runner.hurdle_100m = Medal.Gold
    runner.hurdle_400m = Medal.Silver
    runner.foot_100m = Medal.Gold
    assert runner.medals() == correct, f"Should be: {correct}"

test_api3()